### Задача минимум: по карте пациента выяснить результат лечения
Загружаем необходимые библиотеки

In [1]:
import os
import re

import pymorphy2

Функция удаляет из текста переходы на новую строку, лишние пробелы и приводит слова к нижнему регистру

In [2]:
def format_text(text):
    text = (
        text
        .replace('\n', '')
        .lower()
        .strip()
    )
    text = re.sub(' +', ' ', text)
    return text

Функция удаляет из слова цифры и специальные символы

In [3]:
def format_word(word):
    word = re.sub('[\W\d]', '', word)
    return word

Функция приводит слово к начальной форме

In [4]:
morph = pymorphy2.MorphAnalyzer()
def normalize(word):
    normal_form = morph.parse(word)[0].normal_form
    return normal_form

Функция реализует мешок слов

In [5]:
def count_words(text):
    words_counter = {}
    for word in text.split(' '):
        word = format_word(word)
        if not word:
            continue
        word = normalize(word)
        if word in words_counter:
            words_counter[word] += 1
        else:
            words_counter[word] = 1
    return words_counter

Загружаем текстовый документ. В папке '1' содержится пример, где пациент умер, в '2' - пациент выжил

In [6]:
# path = os.path.join(os.path.abspath('sample from 57'), '1', 'Новый текстовый документ.txt')
path = os.path.join(os.path.abspath('sample from 57'), '2', 'Новый текстовый документ.txt')
with open(path, 'r', encoding='cp1251') as txt_file:
    doc = txt_file.read()

Тексты в предложенных карточках разделены на абзацы, для выяснения результата лечения необходимо найти абзац, который начинается со слов "Состояние при выписке", либо "Дальнейшее течение заболевания"

In [7]:
paragraphs = re.split('\n.\n', doc)

In [8]:
bag_of_words = {}
for paragraph in paragraphs:
    if (paragraph.find('Состояние при выписке') > 0 or 
            paragraph.find('Дальнейшее течение заболевания') > 0):
        paragraph = format_text(paragraph)
        bag_of_words = count_words(paragraph)
        print(paragraph)
        break

состояние при выписке удовлетворительное 1- е сутки после повторного нейролизиса чревного сплетения. состояние больного стабильное. тошноты рвоты нет. болевой синдром купирован. т – 36.6 с. чд – 18 в мин. пульс – 76 в мин. ад – 110/80 мм.рт.ст. язык влажный. живот не вздут мягкий, слегка болезненный в правом подребрье. перитониальных симптомов нет. физ.отправления без особенностей. закл.: течение послеоперационного периода без особенностей. проводилась симптоматическая антибактериальная терапия ,реабилитация больной. выписывается из стационара под наблюдение онколога поликлиники..


Здесь мешок слов выводится в виде pandas.DataFrame() исключительно ради экономии места в тетрадке

In [9]:
import pandas as pd
pd.DataFrame(pd.Series(bag_of_words)).T

,ада,антибактериальный,без,болевой,болезненный,больной,в,вздутый,влажный,выписка,...,сутки,терапия,течение,том,тошнота,удовлетворительный,физотправление,чд,чревный,язык
0,1,1,2,1,1,2,3,1,1,1,...,1,1,1,1,1,1,1,1,1,1


Наивно прикинем результаты лечения

In [10]:
positive_state = False
negative_state = False
if [word for word in ('выписка', 'выписываться', 'удовлетворительный') if word in bag_of_words]:
    positive_state = True
if [word for word in ('смерть', 'реанимация') if word in bag_of_words]:
    negative_state = True
    
if positive_state and negative_state:
    print('Необходим дополнительный анализ')
elif positive_state:
    print('Вероятно, лечение оказало положительный результат')
elif negative_state:
    print('Вероятно, лечение оказало негативный результат')

Вероятно, лечение оказало положительный результат
